## Load Libraries

In [1]:
import warnings
import pandas as pd
import numpy as np
import statsmodels.api as sm

warnings.filterwarnings('ignore')

## Import Cleaned Investigation Data

In [2]:
inv_df = pd.read_csv('Cleaned Investigation Data.csv')

In [3]:
# Check
rows = len(inv_df)
print(f"There are {rows} observations")

There are 1116 observations


## Create Exogenous and Engodenous variables

### 1. Exogenous (Features)

In [4]:
X = inv_df[[
    # General
    'investigation_duration', 
    'time_between_deferral_start_and_investigation', 
    
    # Number of (critical) actions & completed (critical) actions
    'number_of_actions', 
    'actions_completed_count',
    'critical_actions_count', 
    'critical_actions_completed_count',
    
    # Cause type
#     '5whys_cause_type_count',
    'systemic_cause_identified', 
    'physical_cause_identified', 
#     'human_cause_identified',
    
    # Volume
    'deferral_volume_net',
    
    # Years - Exclude 2013
#     'year_2014', 'year_2015', 
#     'year_2016', 'year_2017', 'year_2018', 
    'year_2019', 'year_2020', 'year_2021', 
    'year_2022', 'year_2023',
    
    # Region - Drop 'APAC'
    'AGT','GOMC', 
    'ME', 'NS', 'TT',
    
    # Choke - Drop 'Export'
    'Plant Facility', 'Reservoir', 'Well',
    
    # Investigation Type
    '5 Whys','Cause Map',
]]

In [5]:
X = sm.add_constant(X)

### 2. Endogenous (Target)

### 1. Binarize recurrent counts

In [6]:
columns_to_binarize = ['recurrent_count_1_365', 'recurrent_count_90_365', 'recurrent_count_180_365',
                       'recurrent_count_1_730', 'recurrent_count_90_730', 'recurrent_count_180_730']
inv_df[columns_to_binarize] = (inv_df[columns_to_binarize] > 0).astype(int)

### 2. Define variables for each target

In [7]:
y_1_365 = inv_df['recurrent_count_1_365']
y_90_365 = inv_df['recurrent_count_90_365']
y_180_365 = inv_df['recurrent_count_180_365']
y_1_730 = inv_df['recurrent_count_1_730']
y_90_730 = inv_df['recurrent_count_90_730']
y_180_730 = inv_df['recurrent_count_180_730']

# Regression Models

## 1-365 days

### 1. Log-Odds

In [8]:
model = sm.Logit(y_1_365, X)
result = model.fit(maxiter=1000)

Optimization terminated successfully.
         Current function value: 0.651718
         Iterations 7


In [9]:
summary_table = result.summary()
summary_table

<class 'statsmodels.iolib.summary.Summary'>
"""
                             Logit Regression Results                            
=================================================================================
Dep. Variable:     recurrent_count_1_365   No. Observations:                 1116
Model:                             Logit   Df Residuals:                     1091
Method:                              MLE   Df Model:                           24
Date:                   Mon, 07 Aug 2023   Pseudo R-squ.:                 0.05406
Time:                           15:23:39   Log-Likelihood:                -727.32
converged:                          True   LL-Null:                       -768.88
Covariance Type:               nonrobust   LLR p-value:                 1.913e-08
=================================================================================================================
                                                    coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
const                                            -0.6427      0.702     -0.915      0.360      -2.019       0.733
investigation_duration                           -0.0017      0.001     -2.943      0.003      -0.003      -0.001
time_between_deferral_start_and_investigation    -0.0016      0.001     -3.141      0.002      -0.003      -0.001
number_of_actions                                 0.1546      0.108      1.430      0.153      -0.057       0.367
actions_completed_count                          -0.1914      0.113     -1.700      0.089      -0.412       0.029
critical_actions_count                           -0.1784      0.139     -1.287      0.198      -0.450       0.093
critical_actions_completed_count                 -0.2503      0.611     -0.409      0.682      -1.449       0.948
systemic_cause_identified                        -2.2096      1.290     -1.713      0.087      -4.737       0.318
physical_cause_identified                         2.1612      1.213      1.782      0.075      -0.216       4.539
deferral_volume_net                              -0.0006      0.001     -1.143      0.253      -0.002       0.000
year_2019                                         0.1929      0.185      1.045      0.296      -0.169       0.555
year_2020                                         0.0707      0.205      0.345      0.730      -0.331       0.472
year_2021                                         0.1536      0.264      0.583      0.560      -0.363       0.670
year_2022                                         0.0905      0.275      0.329      0.742      -0.449       0.630
year_2023                                        -0.9817      0.504     -1.948      0.051      -1.969       0.006
AGT                                               0.3675      0.482      0.763      0.445      -0.576       1.311
GOMC                                              0.1458      0.454      0.321      0.748      -0.744       1.036
ME                                               -0.4511      0.741     -0.608      0.543      -1.904       1.002
NS                                                0.7448      0.458      1.626      0.104      -0.153       1.642
TT                                                0.3858      0.500      0.771      0.441      -0.595       1.366
Plant Facility                                    0.6600      0.500      1.320      0.187      -0.320       1.640
Reservoir                                         0.8981      0.989      0.908      0.364      -1.040       2.836
Well                                              0.9035      0.579      1.561      0.118      -0.231       2.038
5 Whys                                           -0.2890      0.259     -1.115      0.265      -0.797       0.219
Cause Map                                        -0.6100      0.712     -0.857      0.391      -2.005       0.785
======================

### 2. Probabilities - at $\mu$

In [10]:
# Get the marginal effects
marginal_effects = result.get_margeff(at='mean')

# Display both the original summary table and the marginal effects summary table
marginal_effects.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
         Logit Marginal Effects         
========================================
Dep. Variable:     recurrent_count_1_365
Method:                             dydx
At:                                 mean
=================================================================================================================
                                                   dy/dx    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
investigation_duration                           -0.0004      0.000     -2.946      0.003      -0.001      -0.000
time_between_deferral_start_and_investigation    -0.0004      0.000     -3.172      0.002      -0.001      -0.000
number_of_actions                                 0.0380      0.027      1.429      0.153      -0.014       0.090
actions_completed_count                          -0.0471      0.028     -1.699      0.089      -0.101       0.007
critical_actions_count                           -0.0439      0.034     -1.287      0.198      -0.111       0.023
critical_actions_completed_count                 -0.0616      0.150     -0.409      0.682      -0.356       0.233
systemic_cause_identified                        -0.5436      0.317     -1.713      0.087      -1.165       0.078
physical_cause_identified                         0.5317      0.298      1.781      0.075      -0.053       1.117
deferral_volume_net                              -0.0002      0.000     -1.143      0.253      -0.000       0.000
year_2019                                         0.0475      0.045      1.045      0.296      -0.042       0.136
year_2020                                         0.0174      0.050      0.345      0.730      -0.081       0.116
year_2021                                         0.0378      0.065      0.583      0.560      -0.089       0.165
year_2022                                         0.0223      0.068      0.329      0.742      -0.110       0.155
year_2023                                        -0.2415      0.124     -1.948      0.051      -0.484       0.001
AGT                                               0.0904      0.118      0.763      0.445      -0.142       0.323
GOMC                                              0.0359      0.112      0.321      0.748      -0.183       0.255
ME                                               -0.1110      0.182     -0.608      0.543      -0.468       0.247
NS                                                0.1832      0.113      1.626      0.104      -0.038       0.404
TT                                                0.0949      0.123      0.771      0.440      -0.146       0.336
Plant Facility                                    0.1624      0.123      1.320      0.187      -0.079       0.403
Reservoir                                         0.2209      0.243      0.908      0.364      -0.256       0.698
Well                                              0.2223      0.142      1.562      0.118      -0.057       0.501
5 Whys                                           -0.0711      0.064     -1.115      0.265      -0.196       0.054
Cause Map                                        -0.1501      0.175     -0.857      0.391      -0.493       0.193
=================================================================================================================
"""

### 3. Probabilities - at 0

In [11]:
# Get the marginal effects
marginal_effects = result.get_margeff(at='zero')

# Display both the original summary table and the marginal effects summary table
marginal_effects.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
         Logit Marginal Effects         
========================================
Dep. Variable:     recurrent_count_1_365
Method:                             dydx
At:                                 zero
=================================================================================================================
                                                   dy/dx    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
investigation_duration                           -0.0004      0.000     -2.402      0.016      -0.001   -6.96e-05
time_between_deferral_start_and_investigation    -0.0004      0.000     -2.636      0.008      -0.001   -9.13e-05
number_of_actions                                 0.0349      0.025      1.384      0.166      -0.015       0.084
actions_completed_count                          -0.0432      0.027     -1.604      0.109      -0.096       0.010
critical_actions_count                           -0.0403      0.032     -1.254      0.210      -0.103       0.023
critical_actions_completed_count                 -0.0565      0.139     -0.408      0.683      -0.328       0.215
systemic_cause_identified                        -0.4991      0.312     -1.600      0.110      -1.111       0.112
physical_cause_identified                         0.4881      0.297      1.644      0.100      -0.094       1.070
deferral_volume_net                              -0.0001      0.000     -1.076      0.282      -0.000       0.000
year_2019                                         0.0436      0.042      1.029      0.304      -0.039       0.127
year_2020                                         0.0160      0.046      0.345      0.730      -0.075       0.107
year_2021                                         0.0347      0.059      0.590      0.555      -0.080       0.150
year_2022                                         0.0204      0.062      0.330      0.741      -0.101       0.142
year_2023                                        -0.2217      0.124     -1.792      0.073      -0.464       0.021
AGT                                               0.0830      0.099      0.841      0.400      -0.110       0.277
GOMC                                              0.0329      0.098      0.335      0.738      -0.160       0.226
ME                                               -0.1019      0.177     -0.575      0.565      -0.449       0.245
NS                                                0.1682      0.087      1.930      0.054      -0.003       0.339
TT                                                0.0871      0.105      0.833      0.405      -0.118       0.292
Plant Facility                                    0.1491      0.094      1.594      0.111      -0.034       0.332
Reservoir                                         0.2029      0.213      0.954      0.340      -0.214       0.620
Well                                              0.2041      0.113      1.812      0.070      -0.017       0.425
5 Whys                                           -0.0653      0.064     -1.013      0.311      -0.192       0.061
Cause Map                                        -0.1378      0.167     -0.827      0.408      -0.464       0.189
=================================================================================================================
"""

## 90-365 days

### 1. Log-Odds

In [12]:
model = sm.Logit(y_90_365, X)
result = model.fit(maxiter=1000)

Optimization terminated successfully.
         Current function value: 0.582402
         Iterations 8


In [13]:
summary_table = result.summary()
summary_table

<class 'statsmodels.iolib.summary.Summary'>
"""
                             Logit Regression Results                             
==================================================================================
Dep. Variable:     recurrent_count_90_365   No. Observations:                 1116
Model:                              Logit   Df Residuals:                     1091
Method:                               MLE   Df Model:                           24
Date:                    Mon, 07 Aug 2023   Pseudo R-squ.:                 0.04918
Time:                            15:23:39   Log-Likelihood:                -649.96
converged:                           True   LL-Null:                       -683.58
Covariance Type:                nonrobust   LLR p-value:                 5.690e-06
=================================================================================================================
                                                    coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
const                                            -0.8720      0.748     -1.166      0.244      -2.338       0.594
investigation_duration                           -0.0011      0.001     -1.770      0.077      -0.002       0.000
time_between_deferral_start_and_investigation    -0.0026      0.001     -3.283      0.001      -0.004      -0.001
number_of_actions                                -0.0275      0.111     -0.247      0.805      -0.246       0.191
actions_completed_count                          -0.0370      0.117     -0.316      0.752      -0.266       0.192
critical_actions_count                            0.0400      0.142      0.281      0.779      -0.239       0.319
critical_actions_completed_count                 -1.1877      0.951     -1.249      0.212      -3.052       0.677
systemic_cause_identified                        -1.5724      1.275     -1.233      0.217      -4.071       0.927
physical_cause_identified                         1.1832      1.113      1.063      0.288      -0.998       3.364
deferral_volume_net                              -0.0006      0.001     -0.927      0.354      -0.002       0.001
year_2019                                         0.3527      0.193      1.823      0.068      -0.026       0.732
year_2020                                         0.2920      0.218      1.339      0.181      -0.135       0.719
year_2021                                         0.5260      0.274      1.918      0.055      -0.012       1.064
year_2022                                         0.3091      0.309      0.999      0.318      -0.297       0.916
year_2023                                        -1.0966      0.659     -1.665      0.096      -2.387       0.194
AGT                                               0.6172      0.518      1.191      0.234      -0.399       1.633
GOMC                                              0.1928      0.491      0.392      0.695      -0.770       1.156
ME                                               -0.1175      0.812     -0.145      0.885      -1.709       1.474
NS                                                0.6626      0.493      1.344      0.179      -0.304       1.629
TT                                                0.3736      0.543      0.689      0.491      -0.690       1.437
Plant Facility                                    0.1842      0.530      0.348      0.728      -0.854       1.222
Reservoir                                         0.4318      1.044      0.413      0.679      -1.615       2.479
Well                                              0.6946      0.612      1.135      0.256      -0.505       1.894
5 Whys                                           -0.3892      0.271     -1.436      0.151      -0.921       0.142
Cause Map                                        -0.7847      0.804     -0.976      0.329      -2.360       0.790
=============

### 2. Probabilities - at $\mu$

In [14]:
# Get the marginal effects
marginal_effects = result.get_margeff(at='mean')

# Display both the original summary table and the marginal effects summary table
marginal_effects.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
          Logit Marginal Effects         
=========================================
Dep. Variable:     recurrent_count_90_365
Method:                              dydx
At:                                  mean
=================================================================================================================
                                                   dy/dx    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
investigation_duration                           -0.0002      0.000     -1.773      0.076      -0.000    2.22e-05
time_between_deferral_start_and_investigation    -0.0005      0.000     -3.543      0.000      -0.001      -0.000
number_of_actions                                -0.0054      0.022     -0.247      0.805      -0.048       0.037
actions_completed_count                          -0.0072      0.023     -0.316      0.752      -0.052       0.037
critical_actions_count                            0.0078      0.028      0.281      0.779      -0.047       0.062
critical_actions_completed_count                 -0.2313      0.184     -1.258      0.208      -0.592       0.129
systemic_cause_identified                        -0.3062      0.248     -1.237      0.216      -0.791       0.179
physical_cause_identified                         0.2304      0.216      1.065      0.287      -0.193       0.654
deferral_volume_net                              -0.0001      0.000     -0.928      0.353      -0.000       0.000
year_2019                                         0.0687      0.038      1.826      0.068      -0.005       0.142
year_2020                                         0.0569      0.042      1.342      0.180      -0.026       0.140
year_2021                                         0.1024      0.053      1.923      0.054      -0.002       0.207
year_2022                                         0.0602      0.060      1.004      0.315      -0.057       0.178
year_2023                                        -0.2135      0.128     -1.670      0.095      -0.464       0.037
AGT                                               0.1202      0.101      1.192      0.233      -0.077       0.318
GOMC                                              0.0375      0.096      0.392      0.695      -0.150       0.225
ME                                               -0.0229      0.158     -0.145      0.885      -0.333       0.287
NS                                                0.1290      0.096      1.344      0.179      -0.059       0.317
TT                                                0.0727      0.106      0.689      0.491      -0.134       0.280
Plant Facility                                    0.0359      0.103      0.348      0.728      -0.166       0.238
Reservoir                                         0.0841      0.203      0.413      0.679      -0.315       0.483
Well                                              0.1353      0.119      1.137      0.255      -0.098       0.368
5 Whys                                           -0.0758      0.053     -1.434      0.152      -0.179       0.028
Cause Map                                        -0.1528      0.157     -0.976      0.329      -0.460       0.154
=================================================================================================================
"""

### 3. Probabilities - at 0

In [15]:
# Get the marginal effects
marginal_effects = result.get_margeff(at='zero')

# Display both the original summary table and the marginal effects summary table
marginal_effects.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
          Logit Marginal Effects         
=========================================
Dep. Variable:     recurrent_count_90_365
Method:                              dydx
At:                                  zero
=================================================================================================================
                                                   dy/dx    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
investigation_duration                           -0.0002      0.000     -1.509      0.131      -0.001    6.74e-05
time_between_deferral_start_and_investigation    -0.0006      0.000     -2.340      0.019      -0.001   -8.95e-05
number_of_actions                                -0.0057      0.023     -0.245      0.806      -0.051       0.040
actions_completed_count                          -0.0077      0.024     -0.315      0.752      -0.056       0.040
critical_actions_count                            0.0083      0.030      0.279      0.781      -0.050       0.067
critical_actions_completed_count                 -0.2469      0.212     -1.167      0.243      -0.662       0.168
systemic_cause_identified                        -0.3269      0.285     -1.149      0.251      -0.885       0.231
physical_cause_identified                         0.2460      0.246      0.999      0.318      -0.236       0.728
deferral_volume_net                              -0.0001      0.000     -0.863      0.388      -0.000       0.000
year_2019                                         0.0733      0.045      1.620      0.105      -0.015       0.162
year_2020                                         0.0607      0.049      1.247      0.212      -0.035       0.156
year_2021                                         0.1094      0.061      1.790      0.073      -0.010       0.229
year_2022                                         0.0643      0.066      0.978      0.328      -0.065       0.193
year_2023                                        -0.2280      0.154     -1.481      0.139      -0.530       0.074
AGT                                               0.1283      0.089      1.443      0.149      -0.046       0.303
GOMC                                              0.0401      0.095      0.421      0.674      -0.146       0.227
ME                                               -0.0244      0.172     -0.142      0.887      -0.361       0.312
NS                                                0.1378      0.085      1.630      0.103      -0.028       0.303
TT                                                0.0777      0.103      0.756      0.449      -0.124       0.279
Plant Facility                                    0.0383      0.102      0.374      0.708      -0.162       0.239
Reservoir                                         0.0898      0.210      0.428      0.668      -0.321       0.500
Well                                              0.1444      0.109      1.320      0.187      -0.070       0.359
5 Whys                                           -0.0809      0.068     -1.188      0.235      -0.214       0.053
Cause Map                                        -0.1632      0.178     -0.915      0.360      -0.513       0.186
=================================================================================================================
"""

# 180-365 days

### 1. Log-Odds

In [16]:
model = sm.Logit(y_180_365, X)
result = model.fit(maxiter=1000)

         Current function value: 0.461947
         Iterations: 1000


In [17]:
summary_table = result.summary()
summary_table

<class 'statsmodels.iolib.summary.Summary'>
"""
                              Logit Regression Results                             
===================================================================================
Dep. Variable:     recurrent_count_180_365   No. Observations:                 1116
Model:                               Logit   Df Residuals:                     1091
Method:                                MLE   Df Model:                           24
Date:                     Mon, 07 Aug 2023   Pseudo R-squ.:                 0.06223
Time:                             15:23:39   Log-Likelihood:                -515.53
converged:                           False   LL-Null:                       -549.75
Covariance Type:                 nonrobust   LLR p-value:                 3.777e-06
=================================================================================================================
                                                    coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
const                                            -1.9019      0.947     -2.009      0.045      -3.758      -0.046
investigation_duration                           -0.0011      0.001     -1.457      0.145      -0.003       0.000
time_between_deferral_start_and_investigation    -0.0015      0.001     -1.899      0.058      -0.003    4.73e-05
number_of_actions                                 0.0343      0.128      0.269      0.788      -0.216       0.284
actions_completed_count                          -0.0766      0.134     -0.573      0.567      -0.339       0.185
critical_actions_count                            0.2344      0.244      0.961      0.337      -0.244       0.713
critical_actions_completed_count                 -0.9332      1.235     -0.755      0.450      -3.354       1.488
systemic_cause_identified                       -33.6394   1.02e+07  -3.29e-06      1.000      -2e+07       2e+07
physical_cause_identified                         0.6694      1.323      0.506      0.613      -1.924       3.263
deferral_volume_net                           -2.539e-05      0.001     -0.042      0.967      -0.001       0.001
year_2019                                         0.1118      0.224      0.498      0.618      -0.328       0.551
year_2020                                         0.1137      0.252      0.451      0.652      -0.380       0.607
year_2021                                         0.4519      0.307      1.472      0.141      -0.150       1.054
year_2022                                        -0.3346      0.393     -0.850      0.395      -1.106       0.437
year_2023                                        -2.0278      1.060     -1.913      0.056      -4.106       0.050
AGT                                               1.0467      0.689      1.520      0.129      -0.303       2.397
GOMC                                              0.5254      0.665      0.790      0.430      -0.778       1.829
ME                                              -26.6228   4.69e+05  -5.67e-05      1.000    -9.2e+05     9.2e+05
NS                                                1.1212      0.664      1.688      0.091      -0.180       2.423
TT                                                0.8196      0.713      1.149      0.251      -0.579       2.218
Plant Facility                                    0.1740      0.645      0.270      0.787      -1.091       1.439
Reservoir                                         0.0446      1.297      0.034      0.973      -2.498       2.587
Well                                              0.8428      0.726      1.162      0.245      -0.579       2.265
5 Whys                                           -0.3291      0.307     -1.072      0.284      -0.931       0.273
Cause Map                                       -39.3974   9.67e+06  -4.07e-06      1.000    -1.9e+07     1.9e+07
====

### 2. Probabilities - at $\mu$

In [18]:
# Get the marginal effects
marginal_effects = result.get_margeff(at='mean')

# Display both the original summary table and the marginal effects summary table
marginal_effects.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
          Logit Marginal Effects          
==========================================
Dep. Variable:     recurrent_count_180_365
Method:                               dydx
At:                                   mean
=================================================================================================================
                                                   dy/dx    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
investigation_duration                        -4.453e-05      9.809  -4.54e-06      1.000     -19.225      19.225
time_between_deferral_start_and_investigation -5.986e-05     13.188  -4.54e-06      1.000     -25.847      25.847
number_of_actions                                 0.0014    307.257   4.54e-06      1.000    -602.211     602.214
actions_completed_count                          -0.0031    687.106  -4.54e-06      1.000   -1346.707    1346.701
critical_actions_count                            0.0095   2103.003   4.54e-06      1.000   -4121.800    4121.820
critical_actions_completed_count                 -0.0380   8370.632  -4.54e-06      1.000   -1.64e+04    1.64e+04
systemic_cause_identified                        -1.3698    2.7e+05  -5.06e-06      1.000    -5.3e+05     5.3e+05
physical_cause_identified                         0.0273   6004.807   4.54e-06      1.000   -1.18e+04    1.18e+04
deferral_volume_net                           -1.034e-06      0.228  -4.54e-06      1.000      -0.446       0.446
year_2019                                         0.0046   1002.878   4.54e-06      1.000   -1965.601    1965.610
year_2020                                         0.0046   1019.962   4.54e-06      1.000   -1999.085    1999.094
year_2021                                         0.0184   4053.624   4.54e-06      1.000   -7944.939    7944.976
year_2022                                        -0.0136   3001.117  -4.54e-06      1.000   -5882.096    5882.069
year_2023                                        -0.0826   1.82e+04  -4.54e-06      1.000   -3.57e+04    3.57e+04
AGT                                               0.0426   9389.568   4.54e-06      1.000   -1.84e+04    1.84e+04
GOMC                                              0.0214   4712.575   4.54e-06      1.000   -9236.455    9236.498
ME                                               -1.0841   2.39e+05  -4.53e-06      1.000   -4.69e+05    4.69e+05
NS                                                0.0457   1.01e+04   4.54e-06      1.000   -1.97e+04    1.97e+04
TT                                                0.0334   7352.241   4.54e-06      1.000   -1.44e+04    1.44e+04
Plant Facility                                    0.0071   1560.994   4.54e-06      1.000   -3059.485    3059.499
Reservoir                                         0.0018    400.261   4.54e-06      1.000    -784.495     784.499
Well                                              0.0343   7560.311   4.54e-06      1.000   -1.48e+04    1.48e+04
5 Whys                                           -0.0134   2951.934  -4.54e-06      1.000   -5785.697    5785.670
Cause Map                                        -1.6042   3.16e+05  -5.08e-06      1.000   -6.19e+05    6.19e+05
=================================================================================================================
"""

### 3. Probabilities - at 0

In [19]:
# Get the marginal effects
marginal_effects = result.get_margeff(at='zero')

# Display both the original summary table and the marginal effects summary table
marginal_effects.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
          Logit Marginal Effects          
==========================================
Dep. Variable:     recurrent_count_180_365
Method:                               dydx
At:                                   zero
=================================================================================================================
                                                   dy/dx    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
investigation_duration                           -0.0001      0.000     -0.991      0.322      -0.000       0.000
time_between_deferral_start_and_investigation    -0.0002      0.000     -1.156      0.248      -0.000       0.000
number_of_actions                                 0.0039      0.015      0.266      0.790      -0.025       0.032
actions_completed_count                          -0.0087      0.016     -0.536      0.592      -0.040       0.023
critical_actions_count                            0.0265      0.034      0.780      0.435      -0.040       0.093
critical_actions_completed_count                 -0.1055      0.158     -0.669      0.503      -0.414       0.203
systemic_cause_identified                        -3.8020   1.16e+06  -3.29e-06      1.000   -2.27e+06    2.27e+06
physical_cause_identified                         0.0757      0.160      0.472      0.637      -0.239       0.390
deferral_volume_net                           -2.869e-06   6.92e-05     -0.041      0.967      -0.000       0.000
year_2019                                         0.0126      0.026      0.477      0.633      -0.039       0.065
year_2020                                         0.0129      0.030      0.433      0.665      -0.045       0.071
year_2021                                         0.0511      0.045      1.123      0.261      -0.038       0.140
year_2022                                        -0.0378      0.053     -0.710      0.478      -0.142       0.067
year_2023                                        -0.2292      0.199     -1.149      0.250      -0.620       0.162
AGT                                               0.1183      0.065      1.807      0.071      -0.010       0.247
GOMC                                              0.0594      0.057      1.046      0.295      -0.052       0.171
ME                                               -3.0090    5.3e+04  -5.67e-05      1.000   -1.04e+05    1.04e+05
NS                                                0.1267      0.069      1.824      0.068      -0.009       0.263
TT                                                0.0926      0.068      1.365      0.172      -0.040       0.226
Plant Facility                                    0.0197      0.065      0.304      0.761      -0.107       0.146
Reservoir                                         0.0050      0.146      0.035      0.972      -0.280       0.290
Well                                              0.0953      0.072      1.324      0.186      -0.046       0.236
5 Whys                                           -0.0372      0.049     -0.766      0.444      -0.132       0.058
Cause Map                                        -4.4528   1.09e+06  -4.07e-06      1.000   -2.14e+06    2.14e+06
=================================================================================================================
"""

# 1-730 days

In [20]:
model = sm.Logit(y_1_730, X)
result = model.fit(maxiter=1000)

Optimization terminated successfully.
         Current function value: 0.647358
         Iterations 7


### 1. Log-Odds

In [21]:
summary_table = result.summary()
summary_table

<class 'statsmodels.iolib.summary.Summary'>
"""
                             Logit Regression Results                            
=================================================================================
Dep. Variable:     recurrent_count_1_730   No. Observations:                 1116
Model:                             Logit   Df Residuals:                     1091
Method:                              MLE   Df Model:                           24
Date:                   Mon, 07 Aug 2023   Pseudo R-squ.:                 0.06557
Time:                           15:23:39   Log-Likelihood:                -722.45
converged:                          True   LL-Null:                       -773.15
Covariance Type:               nonrobust   LLR p-value:                 1.736e-11
=================================================================================================================
                                                    coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
const                                            -0.3957      0.679     -0.583      0.560      -1.726       0.934
investigation_duration                           -0.0016      0.001     -2.952      0.003      -0.003      -0.001
time_between_deferral_start_and_investigation    -0.0021      0.001     -3.887      0.000      -0.003      -0.001
number_of_actions                                 0.1124      0.108      1.038      0.299      -0.100       0.325
actions_completed_count                          -0.1570      0.113     -1.393      0.164      -0.378       0.064
critical_actions_count                           -0.1376      0.137     -1.005      0.315      -0.406       0.131
critical_actions_completed_count                 -0.1729      0.537     -0.322      0.747      -1.225       0.880
systemic_cause_identified                        -1.8499      1.281     -1.444      0.149      -4.361       0.662
physical_cause_identified                         1.7700      1.210      1.462      0.144      -0.602       4.142
deferral_volume_net                              -0.0003      0.000     -0.688      0.492      -0.001       0.001
year_2019                                         0.3506      0.188      1.868      0.062      -0.017       0.718
year_2020                                         0.2523      0.207      1.222      0.222      -0.153       0.657
year_2021                                         0.4220      0.266      1.586      0.113      -0.100       0.944
year_2022                                         0.3844      0.277      1.386      0.166      -0.159       0.928
year_2023                                        -0.7506      0.475     -1.581      0.114      -1.681       0.180
AGT                                               0.7600      0.479      1.588      0.112      -0.178       1.698
GOMC                                              0.3562      0.450      0.792      0.428      -0.525       1.238
ME                                                0.4564      0.692      0.659      0.510      -0.900       1.813
NS                                                1.0550      0.456      2.315      0.021       0.162       1.948
TT                                                0.5124      0.497      1.032      0.302      -0.461       1.486
Plant Facility                                    0.3884      0.466      0.833      0.405      -0.526       1.302
Reservoir                                         1.0548      0.998      1.057      0.291      -0.901       3.011
Well                                              0.6835      0.549      1.246      0.213      -0.392       1.759
5 Whys                                           -0.3591      0.262     -1.370      0.171      -0.873       0.155
Cause Map                                        -1.1601      0.708     -1.640      0.101      -2.547       0.227
======================

### 2. Probabilities - at $\mu$

In [22]:
# Get the marginal effects
marginal_effects = result.get_margeff(at='mean')

# Display both the original summary table and the marginal effects summary table
marginal_effects.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
         Logit Marginal Effects         
========================================
Dep. Variable:     recurrent_count_1_730
Method:                             dydx
At:                                 mean
=================================================================================================================
                                                   dy/dx    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
investigation_duration                           -0.0004      0.000     -2.952      0.003      -0.001      -0.000
time_between_deferral_start_and_investigation    -0.0005      0.000     -3.891      0.000      -0.001      -0.000
number_of_actions                                 0.0281      0.027      1.038      0.299      -0.025       0.081
actions_completed_count                          -0.0392      0.028     -1.392      0.164      -0.094       0.016
critical_actions_count                           -0.0344      0.034     -1.005      0.315      -0.101       0.033
critical_actions_completed_count                 -0.0432      0.134     -0.322      0.747      -0.306       0.220
systemic_cause_identified                        -0.4624      0.320     -1.444      0.149      -1.090       0.165
physical_cause_identified                         0.4424      0.303      1.462      0.144      -0.151       1.035
deferral_volume_net                           -8.592e-05      0.000     -0.688      0.492      -0.000       0.000
year_2019                                         0.0876      0.047      1.868      0.062      -0.004       0.180
year_2020                                         0.0631      0.052      1.222      0.222      -0.038       0.164
year_2021                                         0.1055      0.067      1.586      0.113      -0.025       0.236
year_2022                                         0.0961      0.069      1.386      0.166      -0.040       0.232
year_2023                                        -0.1876      0.119     -1.581      0.114      -0.420       0.045
AGT                                               0.1900      0.120      1.588      0.112      -0.045       0.425
GOMC                                              0.0890      0.112      0.792      0.428      -0.131       0.309
ME                                                0.1141      0.173      0.659      0.510      -0.225       0.453
NS                                                0.2637      0.114      2.315      0.021       0.040       0.487
TT                                                0.1281      0.124      1.032      0.302      -0.115       0.371
Plant Facility                                    0.0971      0.117      0.833      0.405      -0.131       0.326
Reservoir                                         0.2637      0.250      1.057      0.291      -0.225       0.753
Well                                              0.1709      0.137      1.246      0.213      -0.098       0.440
5 Whys                                           -0.0898      0.066     -1.370      0.171      -0.218       0.039
Cause Map                                        -0.2900      0.177     -1.640      0.101      -0.637       0.057
=================================================================================================================
"""

### 3. Probabilities - at 0

In [23]:
# Get the marginal effects
marginal_effects = result.get_margeff(at='mean')

# Display both the original summary table and the marginal effects summary table
marginal_effects.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
         Logit Marginal Effects         
========================================
Dep. Variable:     recurrent_count_1_730
Method:                             dydx
At:                                 mean
=================================================================================================================
                                                   dy/dx    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
investigation_duration                           -0.0004      0.000     -2.952      0.003      -0.001      -0.000
time_between_deferral_start_and_investigation    -0.0005      0.000     -3.891      0.000      -0.001      -0.000
number_of_actions                                 0.0281      0.027      1.038      0.299      -0.025       0.081
actions_completed_count                          -0.0392      0.028     -1.392      0.164      -0.094       0.016
critical_actions_count                           -0.0344      0.034     -1.005      0.315      -0.101       0.033
critical_actions_completed_count                 -0.0432      0.134     -0.322      0.747      -0.306       0.220
systemic_cause_identified                        -0.4624      0.320     -1.444      0.149      -1.090       0.165
physical_cause_identified                         0.4424      0.303      1.462      0.144      -0.151       1.035
deferral_volume_net                           -8.592e-05      0.000     -0.688      0.492      -0.000       0.000
year_2019                                         0.0876      0.047      1.868      0.062      -0.004       0.180
year_2020                                         0.0631      0.052      1.222      0.222      -0.038       0.164
year_2021                                         0.1055      0.067      1.586      0.113      -0.025       0.236
year_2022                                         0.0961      0.069      1.386      0.166      -0.040       0.232
year_2023                                        -0.1876      0.119     -1.581      0.114      -0.420       0.045
AGT                                               0.1900      0.120      1.588      0.112      -0.045       0.425
GOMC                                              0.0890      0.112      0.792      0.428      -0.131       0.309
ME                                                0.1141      0.173      0.659      0.510      -0.225       0.453
NS                                                0.2637      0.114      2.315      0.021       0.040       0.487
TT                                                0.1281      0.124      1.032      0.302      -0.115       0.371
Plant Facility                                    0.0971      0.117      0.833      0.405      -0.131       0.326
Reservoir                                         0.2637      0.250      1.057      0.291      -0.225       0.753
Well                                              0.1709      0.137      1.246      0.213      -0.098       0.440
5 Whys                                           -0.0898      0.066     -1.370      0.171      -0.218       0.039
Cause Map                                        -0.2900      0.177     -1.640      0.101      -0.637       0.057
=================================================================================================================
"""

## 90-730 days

In [24]:
model = sm.Logit(y_90_730, X)
result = model.fit(maxiter=1000)

Optimization terminated successfully.
         Current function value: 0.619796
         Iterations 8


### 1. Log-Odds

In [25]:
summary_table = result.summary()
summary_table

<class 'statsmodels.iolib.summary.Summary'>
"""
                             Logit Regression Results                             
==================================================================================
Dep. Variable:     recurrent_count_90_730   No. Observations:                 1116
Model:                              Logit   Df Residuals:                     1091
Method:                               MLE   Df Model:                           24
Date:                    Mon, 07 Aug 2023   Pseudo R-squ.:                 0.06599
Time:                            15:23:39   Log-Likelihood:                -691.69
converged:                           True   LL-Null:                       -740.56
Covariance Type:                nonrobust   LLR p-value:                 7.263e-11
=================================================================================================================
                                                    coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
const                                            -0.6994      0.694     -1.007      0.314      -2.060       0.662
investigation_duration                           -0.0011      0.001     -1.847      0.065      -0.002    6.52e-05
time_between_deferral_start_and_investigation    -0.0035      0.001     -4.272      0.000      -0.005      -0.002
number_of_actions                                -0.0233      0.102     -0.229      0.819      -0.223       0.176
actions_completed_count                          -0.0151      0.107     -0.142      0.887      -0.224       0.194
critical_actions_count                            0.0186      0.137      0.135      0.892      -0.251       0.288
critical_actions_completed_count                 -0.8612      0.761     -1.131      0.258      -2.353       0.631
systemic_cause_identified                        -0.9523      1.202     -0.792      0.428      -3.308       1.404
physical_cause_identified                         0.4609      1.066      0.432      0.666      -1.629       2.551
deferral_volume_net                           -4.401e-05      0.001     -0.087      0.931      -0.001       0.001
year_2019                                         0.5354      0.188      2.851      0.004       0.167       0.904
year_2020                                         0.4719      0.210      2.248      0.025       0.061       0.883
year_2021                                         0.8896      0.267      3.330      0.001       0.366       1.413
year_2022                                         0.7046      0.295      2.388      0.017       0.126       1.283
year_2023                                        -0.7043      0.547     -1.286      0.198      -1.777       0.369
AGT                                               1.1981      0.502      2.387      0.017       0.214       2.182
GOMC                                              0.5174      0.475      1.090      0.276      -0.413       1.448
ME                                                0.8804      0.717      1.229      0.219      -0.524       2.285
NS                                                1.0692      0.478      2.235      0.025       0.132       2.007
TT                                                0.6440      0.524      1.229      0.219      -0.383       1.671
Plant Facility                                   -0.2519      0.468     -0.538      0.591      -1.170       0.666
Reservoir                                         0.3220      0.966      0.333      0.739      -1.571       2.215
Well                                              0.1879      0.553      0.340      0.734      -0.895       1.271
5 Whys                                           -0.2858      0.261     -1.095      0.273      -0.797       0.226
Cause Map                                        -1.2319      0.792     -1.555      0.120      -2.785       0.321
=============

### 2. Probabilities - at $\mu$

In [26]:
# Get the marginal effects
marginal_effects = result.get_margeff(at='mean')

# Display both the original summary table and the marginal effects summary table
marginal_effects.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
          Logit Marginal Effects         
=========================================
Dep. Variable:     recurrent_count_90_730
Method:                              dydx
At:                                  mean
=================================================================================================================
                                                   dy/dx    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
investigation_duration                           -0.0002      0.000     -1.849      0.064      -0.000    1.42e-05
time_between_deferral_start_and_investigation    -0.0008      0.000     -4.606      0.000      -0.001      -0.000
number_of_actions                                -0.0052      0.023     -0.229      0.819      -0.049       0.039
actions_completed_count                          -0.0033      0.024     -0.142      0.887      -0.050       0.043
critical_actions_count                            0.0041      0.030      0.135      0.892      -0.055       0.064
critical_actions_completed_count                 -0.1907      0.168     -1.136      0.256      -0.520       0.138
systemic_cause_identified                        -0.2109      0.266     -0.793      0.428      -0.732       0.310
physical_cause_identified                         0.1021      0.236      0.432      0.665      -0.361       0.565
deferral_volume_net                           -9.746e-06      0.000     -0.087      0.931      -0.000       0.000
year_2019                                         0.1186      0.042      2.854      0.004       0.037       0.200
year_2020                                         0.1045      0.046      2.254      0.024       0.014       0.195
year_2021                                         0.1970      0.059      3.339      0.001       0.081       0.313
year_2022                                         0.1560      0.065      2.410      0.016       0.029       0.283
year_2023                                        -0.1560      0.121     -1.286      0.198      -0.394       0.082
AGT                                               0.2653      0.111      2.390      0.017       0.048       0.483
GOMC                                              0.1146      0.105      1.090      0.276      -0.091       0.321
ME                                                0.1950      0.159      1.229      0.219      -0.116       0.506
NS                                                0.2368      0.106      2.235      0.025       0.029       0.444
TT                                                0.1426      0.116      1.231      0.218      -0.084       0.370
Plant Facility                                   -0.0558      0.104     -0.538      0.591      -0.259       0.148
Reservoir                                         0.0713      0.214      0.333      0.739      -0.348       0.490
Well                                              0.0416      0.122      0.340      0.734      -0.198       0.281
5 Whys                                           -0.0633      0.058     -1.094      0.274      -0.177       0.050
Cause Map                                        -0.2728      0.175     -1.555      0.120      -0.617       0.071
=================================================================================================================
"""

### 3. Probabilities - at 0

In [27]:
# Get the marginal effects
marginal_effects = result.get_margeff(at='zero')

# Display both the original summary table and the marginal effects summary table
marginal_effects.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
          Logit Marginal Effects         
=========================================
Dep. Variable:     recurrent_count_90_730
Method:                              dydx
At:                                  zero
=================================================================================================================
                                                   dy/dx    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
investigation_duration                           -0.0002      0.000     -1.647      0.100      -0.001     4.5e-05
time_between_deferral_start_and_investigation    -0.0008      0.000     -3.067      0.002      -0.001      -0.000
number_of_actions                                -0.0052      0.023     -0.228      0.819      -0.050       0.039
actions_completed_count                          -0.0034      0.024     -0.142      0.887      -0.050       0.043
critical_actions_count                            0.0041      0.031      0.135      0.893      -0.056       0.064
critical_actions_completed_count                 -0.1910      0.174     -1.096      0.273      -0.532       0.150
systemic_cause_identified                        -0.2112      0.272     -0.777      0.437      -0.744       0.321
physical_cause_identified                         0.1022      0.239      0.428      0.668      -0.365       0.570
deferral_volume_net                            -9.76e-06      0.000     -0.087      0.931      -0.000       0.000
year_2019                                         0.1187      0.049      2.414      0.016       0.022       0.215
year_2020                                         0.1047      0.052      2.006      0.045       0.002       0.207
year_2021                                         0.1973      0.068      2.904      0.004       0.064       0.330
year_2022                                         0.1563      0.072      2.167      0.030       0.015       0.298
year_2023                                        -0.1562      0.127     -1.234      0.217      -0.404       0.092
AGT                                               0.2657      0.085      3.108      0.002       0.098       0.433
GOMC                                              0.1147      0.091      1.267      0.205      -0.063       0.292
ME                                                0.1952      0.144      1.353      0.176      -0.088       0.478
NS                                                0.2371      0.084      2.829      0.005       0.073       0.401
TT                                                0.1428      0.102      1.396      0.163      -0.058       0.343
Plant Facility                                   -0.0559      0.113     -0.495      0.621      -0.277       0.165
Reservoir                                         0.0714      0.210      0.340      0.734      -0.340       0.482
Well                                              0.0417      0.118      0.353      0.724      -0.189       0.273
5 Whys                                           -0.0634      0.064     -0.988      0.323      -0.189       0.062
Cause Map                                        -0.2732      0.192     -1.424      0.155      -0.649       0.103
=================================================================================================================
"""

## 180-730

In [28]:
model = sm.Logit(y_180_730, X)
result = model.fit(maxiter=1000)

         Current function value: inf
         Iterations: 1000


LinAlgError: Singular matrix

### 1. Log-Odds

In [ ]:
summary_table = result.summary()
summary_table

### 2. Probabilities - at $\mu$

In [ ]:
# Get the marginal effects
marginal_effects = result.get_margeff(at='mean')

# Display both the original summary table and the marginal effects summary table
marginal_effects.summary()

### 3. Probabilities - at 0

In [ ]:
# Get the marginal effects
marginal_effects = result.get_margeff(at='zero')

# Display both the original summary table and the marginal effects summary table
marginal_effects.summary()